In [1]:
%load_ext autoreload
%autoreload

import torch
from OScINN import OScINN1D


run a small test on a cinn with random data to test the functionality

In [4]:
oscinn = OScINN1D(9, 100, 8, cuda=False) # create a cINN with input dim:9, conditional input dim:100 and 8 all_in_one_blocks
device = 'cuda' if torch.cuda.is_available() else 'cpu'
c = torch.randn(50, 1, 100).to(device) # create some artificial data to be used as conditional input
x = torch.randn(50, 9).to(device) # create some artificial data to be used as input to the invertible network

c = oscinn.cond_net(c)
print('condition shape: ', c.shape)
c = [c.squeeze() for i in range(8)] # the output of the conditional network should be given to every all_in_one_blocks, this is done by putting all conditional inputs in a list
z, jac_z = oscinn.cinn(x, c=c, rev=False) # output the latents space vector and the log jacobian determinant
print('latent space shape: ', z.shape)

condition shape:  torch.Size([50, 1, 15])
latent space shape:  torch.Size([50, 9])


Lets test training of a cINN on some arbitrary data

In [5]:
c = torch.randn(50, 1, 100).to(device) # create some artificial data to be used as conditional input
x = torch.randn(50, 9).to(device) # create some artificial data to be used as input to the invertible network
# create some dataset to train on
dataset = torch.utils.data.TensorDataset(x, c[:,0])
dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=5,
                                        shuffle=True, 
                                        drop_last=False)
oscinn.optimizer = torch.optim.Adam # sets an optimizer for both the invertible and the conditional neural network
oscinn.optimizer_kwargs = {'lr':0.001}

oscinn.train(dataloader, 2) # the train methods takes the dataloader and the number of epochs that should be trained 


100%|██████████| 2/2 [00:04<00:00,  2.16s/it]


By creating a new dataset and dataloader, the OScINN can be trained of 1D data